In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import nltk 
nltk.download('gutenberg')
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import nltk #the Natural Language Toolkit
import pandas as pd #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import wordcloud #Makes word clouds
import numpy as np #For divergences/distances
import scipy #For divergences/distances
import seaborn as sns #makes our plots look nicer
import sklearn.manifold #For a manifold plot
from nltk.corpus import stopwords #For stopwords
import json #For API responses
import urllib.parse #For joining urls

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook
%matplotlib inline

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Timot\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
anthro_df = pd.read_pickle(r'C:\Users\Timot\Documents\final_project\anthro_data\anthro_norm_filt_df.pk1')
econ_df = pd.read_pickle(r'C:\Users\Timot\Documents\final_project\econ_data\econ_norm_filt_df.pk1')
ling_df = pd.read_pickle(r'C:\Users\Timot\Documents\final_project\ling_data\ling_norm_filt_df.pk1')
soc_df = pd.read_pickle(r'C:\Users\Timot\Documents\final_project\soc_data\soc_norm_filt_df.pk1')

In [ ]:
anthroWords = set(anthro_df['normalized_tokens'].sum())
econWords = set(econ_df['normalized_tokens'].sum())
lingWords = set(ling_df['normalized_tokens'].sum())
socWords = set(soc_df['normalized_tokens'].sum())

#Change & to | if you want to keep all words
overlapWords = anthroWords & econWords & lingWords & socWords

overlapWordsDict = {word: index for index, word in enumerate(overlapWords)}
overlapWordsDict['student']

In [ ]:

def makeProbsArray(dfColumn, overlapDict):
    words = dfColumn.sum()
    countList = [0] * len(overlapDict)
    for word in words:
        try:
            countList[overlapDict[word]] += 1
        except KeyError:
            #The word is not common so we skip it
            pass
    countArray = np.array(countList)
    return countArray / countArray.sum()

whProbArray = makeProbsArray(whReleases['normalized_tokens'], overlapWordsDict)
kenProbArray = makeProbsArray(kenReleases['normalized_tokens'], overlapWordsDict)
kenProbArray.sum()
#There is a little bit of a floating point math error
#but it's too small to see with print and too small matter here

In [ ]:
#computes divergence
wh_kenDivergence = scipy.stats.entropy(whProbArray, kenProbArray)
print (wh_kenDivergence)
ken_whDivergence = scipy.stats.entropy(kenProbArray, whProbArray)
print (ken_whDivergence)

In [ ]:
#displays the words the most distinguish the corpora
wh_kenDivergence_ew = scipy.special.kl_div(whProbArray, kenProbArray)
kl_df = pandas.DataFrame(list(overlapWordsDict.keys()), columns = ['word'], index = list(overlapWordsDict.values()))
kl_df = kl_df.sort_index()
kl_df['elementwise divergence'] = wh_kenDivergence_ew
kl_df[:10]